# Kristins Alex & VGG16 

## Imports

In [1]:
import os
from datetime import datetime
from dataclasses import dataclass

import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import numpy as np

from numpy.random import seed
import pandas as pd
import tensorflow as tf
import random
import shutil

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.initializers import he_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess_input
from tensorflow.keras import Model

available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  1 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13242270196573039188
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 20264236482
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10422467728541766481
physical_device_desc: "device: 0, name: GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
]
1 Physical GPUs, 1 Logical GPUs


In [2]:
def set_seed(s = 1):
    os.environ['PYTHONHASHSEED']='0'

    seed(s)
    tf.random.set_seed(s)
    random.seed(s)
    np.random.seed(s)
set_seed()

## AlexNet

In [3]:
def alexnet(activation, leaky_alpha, dropout):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation=activation_layer,kernel_initializer = he_uniform(seed = 1), input_shape=(224,224,Global.num_image_channels)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation=activation_layer, kernel_initializer = he_uniform(seed = 1),padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=activation_layer, kernel_initializer = he_uniform(seed = 1), padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation=activation_layer, kernel_initializer = he_uniform(seed = 1), padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation=activation_layer, kernel_initializer = he_uniform(seed = 1), padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(units = 2, activation=activation_layer)
        #Dense(10, activation='softmax')
    ])
    return model

## VGG16

In [4]:
def vgg16_old(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    
    model.add(Conv2D(input_shape=(224,224,Global.num_image_channels), filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = activation_layer))
    model.add(Dense(units = 4096, activation = activation_layer))
    #model.add(Dense(units = 2, activation = "softmax"))
    #model.add(Dense(units = 2, activation=activation_layer))
    model.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))

    #opt = Adam(lr = 0.001)
    #model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy, metrics = ['accuracy'])
    #model.summary()
    return model

In [5]:
def vgg16_like_Res(activation, leaky_alpha):
    
    K.clear_session()
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

In [6]:
def vgg16(activation, leaky_alpha, dropout_rate, first_neuron, hidden_layers):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    
    model.add(cnn)

    
    fc = Sequential()
    fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512)
    
    fc.add(Dense(units = first_neuron, kernel_initializer = glorot_uniform(seed = 1)))
    fc.add(activation_layer)
    if dropout_rate > 0.0:
        fc.add(Dropout(rate = dropout_rate))
    
    print('Number Hidden Layers {}'.format(hidden_layers))
    hidden_neuron_fraction = first_neuron
    for i in range(hidden_layers):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        fc.add(activation_layer)
        if dropout_rate > 0.0:
            fc.add(Dropout(rate = dropout_rate))
    
    fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    model.add(fc)
    
    for layer in model.layers:
        layer.trainable = True
    
    return model

## MobileNet

In [7]:
def mobilenet(activation, leaky_alpha):
         
    activation_layer = ReLU()
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
        
    model = Sequential()
    cnn = MobileNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x =  AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## ResNet

In [8]:
def resnet(activation, leaky_alpha):
         
    activation_layer = ReLU()
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
        
    model = Sequential()
    cnn = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x = AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## Generation of the DNN

In [9]:
def gen_net(x_train, y_train, x_val, y_val, params):
    
    K.clear_session()
    set_seed()
    
    train_generator, valid_generator = create_data_pipline(params['batch_size'], params['samples'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
    if(Global.net_architecture == 'ALEX'):
        model = alexnet(params['activation'], params['leaky_alpha'], params['dropout'])
    elif(Global.net_architecture =='VGG16'):
        model = vgg16(params['activation'], params['leaky_alpha'], params['dropout'], params['first_neuron'], params['hidden_layers'])
        #model = vgg16(params['activation'], params['leaky_alpha'])
    elif(Global.net_architecture == 'RESNET'):
        model = resnet(params['leaky_alpha'], params['leaky_alpha'])
    elif(Global.net_architecture == 'MOBILENET'):
        model = mobilenet(params['leaky_alpha'], params['leaky_alpha'])
    else:
        print('Wrong net architecture!')
    model.compile(
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer'])), 
        loss = Global.loss_function, 
        metrics = get_reduction_metric(Global.reduction_metric)
    )
    print(model.summary())
    print('_________________________________________________________________')
    
    checkpointer = callbacks.ModelCheckpoint(
        filepath = Global.target_dir + 'CNN_{}_Model_and_Weights_{}.hdf5'.format(Global.net_architecture, Global.image_channels),
        monitor =  Global.monitor_value,
        verbose = 1,
        save_weights_only = False,
        save_best_only = True,
        mode = 'min'
    )
    print('Checkpointer was created')
    
    csv_logger = callbacks.CSVLogger(
        filename = Global.target_dir + 'CNN_{}_Logger_{}.csv'.format(Global.net_architecture, Global.image_channels),
        separator = ',',
        append = False
    )
    print('CSV Logger was created')

    lr_reducer = callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.1,
        patience = 13,
        verbose = 1,
        mode = 'min',
        min_delta = 0.0001
    )
    print('Learning Rate Reducer was created')
    
    early_stopper = callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0,
        patience = 15,
        #patience = 20,
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    print('Early Stopper was created')
    
    tb_name = 'tb_{}_{}'.format(Global.net_architecture, Global.image_channels)
    tensorboard = callbacks.TensorBoard(log_dir = Global.target_dir + tb_name)
    
    startTime = datetime.now()
    print(startTime)
    
    out = model.fit(
        x = train_generator,
        epochs = params['epochs'],
        validation_data = valid_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_steps = vg_validation_steps,
        callbacks = [checkpointer, csv_logger, lr_reducer, early_stopper, tensorboard],
        workers = 8
    )
    print("Time taken:", datetime.now() - startTime)

    return out, model

### Benutzerdefinierte Kostenfunktion & Metrik

In [10]:
def mean_absolut_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Hilfsfunktion

In [11]:
def get_reduction_metric(metric):
    
    if metric == 'mean_absolut_error':
        return [mean_absolut_error]
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Automatische Optimizer Generierung aus String

In [12]:
def make_optimizer(optimizer):
    # [Adam, Nadam, Adagrad, RMSprop]
    if optimizer == "<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>":
        return Adam
    else:
        print('ERROR::: Unspecified Optimizer')

### Struct for global parameter

In [13]:
@dataclass
class global_parameter:
    #========================================================
    # just change this, everything else will automaticlly adjusted
    
    net_architecture = 'ALEX' # 'ALEX' vs 'VGG16' vs 'RESNET'
    image_channels: str = 'rgb' # 'rgb' vs 'rgba'
    #======================================================== 
    loss_function: str = 'mean_squared_error'
    reduction_metric: str = 'mean_absolut_error'
    monitor_value: str = 'val_mean_absolut_error'
    
    dataset: str = '201129_2031'
    device: str = 'RTX_3090'
    data_augmentation: bool = True
    image_dir: str = '..\\..\\data_generation\\dataset\\{}\\'.format(dataset)
    
    # Default values for rgb -> later ajusted when rgba
    run: str = datetime.now().strftime('%Y%m%d_%H%M')
    num_image_channels: int = 3
    csv_file_name: str = 'labels_ks_RGB.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)

        
Global = global_parameter

if(Global.image_channels == 'rgba'):
    Global.num_image_channels = 4
    Global.csv_file_name: str = 'labels_ks_RGBD.csv'
    Global.csv_file: str = Global.image_dir + Global.csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(Global.net_architecture, Global.dataset, Global.image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(Global.net_architecture, Global.dataset)

### Generierung Datenpipeline (Angepasst für Talos)

In [14]:
def create_data_pipline(batch_size, num_samples):
        
    df = pd.read_csv(Global.csv_file)
    df_shuffled = df.sample(frac = 1, random_state = 1)
    df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
    df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
    
    if Global.data_augmentation:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255,
            #preprocessing_function=vgg16_preprocess_input,
            #preprocessing_function=mobilenet_preprocess_input,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.5, 1.0), 
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
            #preprocessing_function=vgg16_preprocess_input,
            #preprocessing_function=mobilenet_preprocess_input,
        )
        
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = True,
        seed = 1,
        batch_size = batch_size
    )
        
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
        #preprocessing_function=vgg16_preprocess_input,
        #preprocessing_function=mobilenet_preprocess_input,
    )
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = False,
        seed = 1,
        batch_size = batch_size
    )
    
    return train_generator, valid_generator

In [15]:
if(not os.path.exists(Global.target_dir)):
    os.makedirs(Global.target_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(Global.target_dir))

device_file = open(Global.target_dir + '{}.txt'.format(Global.device), "a+")

Directory >>| ..\output\ALEX_201129_2031_rgb\ |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)


### Top Ergebnisse

In [16]:
base_results = Global.target_dir + Global.results
df = pd.read_csv(base_results).drop(columns = ['round_epochs', 'samples', 'epochs'], axis = 0)
sort_value = 'val_mean_absolut_error'
df = df.sort_values(sort_value, axis = 0, ascending = True, inplace = False, kind = 'quicksort', na_position = 'last')
print('Displaying: {}'.format(base_results))
df.head(10)

Displaying: ..\output\ALEX_201129_2031_rgb\\..\ALEX_201129_2031_Results.csv


,Unnamed: 0,start,end,duration,loss,mean_absolut_error,val_loss,val_mean_absolut_error,activation,batch_size,dropout,first_neuron,hidden_layers,leaky_alpha,lr,optimizer
109,37,11/30/20-130851,11/30/20-131006,74.377089,5479.725683,53.844021,2503.251654,30.018900,relu,16,0.25,1024,1,0.1,2,<class 'tensorflow.python.keras.optimizer_v2.a...
322,34,11/30/20-180411,11/30/20-180526,75.030112,5470.871012,47.489311,2585.396397,30.652834,leakyrelu,32,0.50,4096,4,0.1,2,<class 'tensorflow.python.keras.optimizer_v2.a...
346,58,11/30/20-183412,11/30/20-183528,75.873269,6006.158625,43.072105,2656.543782,30.797106,relu,32,0.25,2048,4,0.1,2,<class 'tensorflow.python.keras.optimizer_v2.a...
147,3,11/30/20-140156,11/30/20-140308,72.442904,4277.776667,42.335674,2715.662221,31.111271,leakyrelu,16,0.25,2048,2,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...
277,61,11/30/20-170058,11/30/20-170213,75.321783,4963.850674,44.226074,2514.001707,31.146908,relu,32,0.25,4096,3,0.1,2,<class 'tensorflow.python.keras.optimizer_v2.a...
3,3,11/30/20-105013,11/30/20-105129,75.235178,4222.016681,42.280361,2621.701853,31.213018,leakyrelu,16,0.25,2048,0,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...
291,3,11/30/20-172444,11/30/20-172558,74.926584,4234.765561,42.454365,2383.671189,31.477369,leakyrelu,16,0.25,2048,4,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...
144,0,11/30/20-135816,11/30/20-135930,74.104424,4040.911918,41.191616,2305.360718,31.767559,leakyrelu,16,0.25,1024,2,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...
186,42,11/30/20-145040,11/30/20-145155,74.236727,4465.015001,43.583923,2728.702388,31.865591,relu,16,0.25,4096,2,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...
282,66,11/30/20-170719,11/30/20-170835,75.676169,4440.256610,43.743534,2965.059381,32.052422,relu,32,0.50,2048,3,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...


### GridSerach

#### Hyper Parameter

In [17]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad

global_hyper_parameter = {
    'samples': None,
    'epochs': None,
    'batch_size': None,
    'optimizer': None,
    'lr': None,
    'first_neuron': None,
    'dropout': None,
    'activation': None,
    'leaky_alpha': None,
    'hidden_layers': None,
}


In [18]:
def get_params(top_results_index):
    
    #     Adam = RMSprop + Momentum (lr=0.001)
    #     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
    #     RMSprop = (lr=0.001)
    #     SGD = (lr=0.01)
    #     Adagrad

    hyper_parameter = global_hyper_parameter

    hyper_parameter['samples'] = [100000]
    hyper_parameter['epochs'] = [400]
    hyper_parameter['batch_size'] = [df.iloc[top_results_index]['batch_size']]
    hyper_parameter['optimizer'] = [make_optimizer(df.loc[top_results_index]['optimizer'])]
    hyper_parameter['lr'] = [df.iloc[top_results_index]['lr']]
    hyper_parameter['first_neuron'] = [df.iloc[top_results_index]['first_neuron']]
    hyper_parameter['dropout'] = [df.iloc[top_results_index]['dropout']]
    hyper_parameter['activation'] = [df.iloc[top_results_index]['activation']]
    hyper_parameter['leaky_alpha'] = [0.1] #Default bei LeakyReLU, sonst PReLU
    hyper_parameter['hidden_layers'] = [df.iloc[top_results_index]['hidden_layers']]
    
    return hyper_parameter

### Test to see if Image

In [19]:
from PIL import Image 
#print(Global.csv_file)
df2 = pd.read_csv(Global.csv_file)
print(df2.head(5))
img_nm = df2.iloc[0]['Filename']
#Image(Global.image_dir + img_nm)

img = Image.open(Global.image_dir + img_nm)
img.show()

for i in range(224):
    for j in range(224):
        colors = img.getpixel((i,j))
        print(colors)

                                 Filename  Azimuth  Elevation
0   buddha/rgb/buddha00000000-0-5-0-5.png        0          5
1  buddha/rgb/buddha00000001-0-5-0-10.png        0         10
2  buddha/rgb/buddha00000002-0-5-0-15.png        0         15
3  buddha/rgb/buddha00000003-0-5-0-20.png        0         20
4  buddha/rgb/buddha00000004-0-5-0-25.png        0         25
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(

...

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(4

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(4

### Start Talos

In [ ]:
dummy_x = np.empty((1, 2, Global.num_image_channels, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    top_results_index = 0

    tmp_dir = '..\\{}_TMP_TALOS_{}_{}'.format(Global.run, Global.image_channels, Global.net_architecture)
    trained_results = Global.target_dir + 'Talos_Results_Fine_Idx_{}_{}.csv'.format(Global.net_architecture, Global.image_channels)

    startTime = datetime.now()

    parameters = get_params(top_results_index)

    t = ta.Scan(
        x = dummy_x,
        y = dummy_y,
        model = gen_net,
        params = parameters,
        experiment_name = tmp_dir,
        #shuffle=False,
        reduction_metric = get_reduction_metric(Global.reduction_metric),
        disable_progress_bar = False,
        print_params = True,
        clear_session = True
    )

    print("Time taken:", datetime.now() - startTime)

    #print('Writing Device File')
    #device_file.write('Trained Model: {}'.format(Global.image_channels))
    #device_file.close()
    
    df_experiment_results = pd.read_csv(tmp_dir + '\\' + os.listdir(tmp_dir)[0])
    df_experiment_results['Base'] = None
    for i in range(df_experiment_results.shape[0]):
        df_experiment_results['Base'][i] = Global.image_channels

    if os.path.isfile(trained_results):
        df_experiment_results.to_csv(trained_results, mode = 'a', index = False, header = False)
    else:
        df_experiment_results.to_csv(trained_results, index = False)   

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{'activation': 'relu', 'batch_size': 16, 'dropout': 0.25, 'epochs': 400, 'first_neuron': 1024, 'hidden_layers': 1, 'leaky_alpha': 0.1, 'lr': 2, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 100000}
Found 80000 validated image filenames.
Found 20000 validated image filenames.
Steps per Epoch: 5000, Validation Steps: 1250
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_1 (Con

In [ ]:
shutil.rmtree(tmp_dir)